# Regression models for pile driveability

In the notebook, a Support Vector Regressor will be trained using the RBF (Radial Basis Function). This example is included to show that the mathematical details of a machine learning model are sometimes beyond the abilities of data scientist but a basic insight in the internal workings of the model can still be useful.

Details on the RBF kernel can be found here (https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.RBF.html) with an example of several kernel functions given here (https://scikit-learn.org/stable/auto_examples/svm/plot_svm_regression.html?highlight=svr).

Support vector regression (https://scikit-learn.org/stable/modules/svm.html#svm-regression) tries to draw linear boundaries in the parameter space but the kernel functions allow non-linear boundaries to be created by transformation of the parameter space.

## Package imports

A number of Python packages are required. Numpy, Pandas and Plotly are know from the previous tutorial.  scikit-learn is a comprehensive Python package for machine learning which will be used here.

In [1]:
import pandas as pd
import numpy as np
import sklearn
from plotly import subplots
import plotly.graph_objs as go
from copy import deepcopy

In [2]:
pd.set_option('display.max_columns', 200)

## Pile driving data

### Data import

The same data used before can be imported:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv("/content/drive/MyDrive/Pile driving Kaggle/isfog2020-pile-driving-predictions/training_data.csv")  # Store the contents of the csv file in the variable 'data'
data.head()

,z [m],qc [MPa],fs [MPa],u2 [MPa],ID,Location ID,Blowcount [Blows/m],Normalised ENTRHU [-],Normalised hammer energy [-],Number of blows,Diameter [m],Bottom wall thickness [mm],Pile penetration [m]
0,0.5,5.150400,0.031200,0.006400,EK__0_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0
1,1.0,11.681000,0.082700,0.015400,EK__1_0,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0
2,1.5,11.107600,0.101300,0.019100,EK__1_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0
3,2.0,10.449700,0.127000,0.030200,EK__2_0,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0
4,2.5,10.585762,0.113588,-0.116242,EK__2_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0


In [5]:
data2 = pd.read_csv("/content/drive/MyDrive/Pile driving Kaggle/isfog2020-pile-driving-predictions/training_data_withnormalised.csv")
data2.head()

,z [m],qc [MPa],fs [MPa],u2 [MPa],ID,Location ID,Blowcount [Blows/m],Normalised ENTRHU [-],Normalised hammer energy [-],Number of blows,Diameter [m],Bottom wall thickness [mm],Pile penetration [m],area ratio [-],Push,Total unit weight [kN/m3],Layer no,Vertical total stress [kPa],Water pressure [kPa],Vertical effective stress [kPa],qt [MPa],Delta u2 [MPa],Rf [%],Bq [-],Qt [-],Fr [%],qnet [MPa],Ic [-]
0,0.0,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,1.0,0.0,0.000,0.000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,0.5,5.1504,0.0312,0.0064,EK__0_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,9.5,5.125,4.375,5.152000,0.001275,0.605590,0.000248,1175.428571,0.606709,5.142500,1.536576
2,1.0,11.6810,0.0827,0.0154,EK__1_0,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,19.0,10.250,8.750,11.684850,0.005150,0.707754,0.000441,1333.240000,0.708907,11.665850,1.447213
3,1.5,11.1076,0.1013,0.0191,EK__1_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,28.5,15.375,13.125,11.112375,0.003725,0.911596,0.000336,844.485714,0.913940,11.083875,1.564459
4,2.0,10.4497,0.1270,0.0302,EK__2_0,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,38.0,20.500,17.500,10.457250,0.009700,1.214468,0.000931,595.385714,1.218898,10.419250,1.689489


In [6]:
data2.columns

Index(['z [m]', 'qc [MPa]', 'fs [MPa]', 'u2 [MPa]', 'ID', 'Location ID',
       'Blowcount [Blows/m]', 'Normalised ENTRHU [-]',
       'Normalised hammer energy [-]', 'Number of blows', 'Diameter [m]',
       'Bottom wall thickness [mm]', 'Pile penetration [m]', 'area ratio [-]',
       'Push', 'Total unit weight [kN/m3]', 'Layer no',
       'Vertical total stress [kPa]', 'Water pressure [kPa]',
       'Vertical effective stress [kPa]', 'qt [MPa]', 'Delta u2 [MPa]',
       'Rf [%]', 'Bq [-]', 'Qt [-]', 'Fr [%]', 'qnet [MPa]', 'Ic [-]'],
      dtype='object')

In [7]:
#Adding additional features to data2 that will be used for the calculation of SRD

#Calculation of constant k
data2['k'] = (data2['qc [MPa]']*1000 / data2['Vertical effective stress [kPa]'])**0.5 / 80


In [8]:
#Calculation of the peak shaft friction fsmax


# Define the constants
Patm = 100
tan_delta = 0.5317

# Assuming 'data2' is your DataFrame

qc = 1000 * data2['qc [MPa]'].values
vertical_effective_stress = data2['Vertical effective stress [kPa]'].values

# Calculate 'fsmax' using NumPy
fsmax = (0.0132 * qc *(vertical_effective_stress / Patm)**0.13) * tan_delta

# Add 'fsmax' as a new column to the DataFrame
data2['fsmax'] = fsmax

In [9]:
#Calculation of the residual shaft friction
fsmax = data2['fsmax'].values

# Calculate 'fsres' using NumPy
fsres = 0.2 * fsmax

# Add 'fsres' as a new column to the DataFrame
data2['fsres'] = fsres

In [10]:
#Calculation of the ultimate shaft friction fs
data2['h'] = data2['Pile penetration [m]'] - data2['z [m]']

# Define variables for the formula
fsmax = data2['fsmax'].values
fsres = data2['fsres'].values
k = data2['k'].values
h = data2['h'].values

# Calculate 'fs' using NumPy based on the formula
fs = fsres + (fsmax - fsres) * np.exp(-k * h)

# Add 'fs' as a new column to the DataFrame
data2['fs'] = fs

In [11]:
qc = data2['qc [MPa]'].values
vertical_effective_stress = data2['Vertical effective stress [kPa]'].values

# Calculate 'qaa' using NumPy based on the formula
qaa = 0.15 * qc *(qc*1000 / vertical_effective_stress)**0.2

# Add 'qaa' as a new column to the DataFrame
data2['qaa'] = qaa

<ipython-input-11-2744245a133e>:5: RuntimeWarning: invalid value encountered in divide
  qaa = 0.15 * qc *(qc*1000 / vertical_effective_stress)**0.2


In [12]:
qaaf = data2['qaa'].values
D = data2['Diameter [m]'].values
t_mm = data2['Bottom wall thickness [mm]'].values

# Convert thickness from mm to meters
t = t_mm / 1000  # Convert from mm to meters

# Calculate 'Qb' using NumPy based on the formula
Qb = qaaf * np.pi * D * t

# Add 'Qb' as a new column to the DataFrame
data2['Qb'] = Qb

In [13]:
data2.head()

,z [m],qc [MPa],fs [MPa],u2 [MPa],ID,Location ID,Blowcount [Blows/m],Normalised ENTRHU [-],Normalised hammer energy [-],Number of blows,Diameter [m],Bottom wall thickness [mm],Pile penetration [m],area ratio [-],Push,Total unit weight [kN/m3],Layer no,Vertical total stress [kPa],Water pressure [kPa],Vertical effective stress [kPa],qt [MPa],Delta u2 [MPa],Rf [%],Bq [-],Qt [-],Fr [%],qnet [MPa],Ic [-],k,fsmax,fsres,h,fs,qaa,Qb
0,0.0,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,1.0,0.0,0.000,0.000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
1,0.5,5.1504,0.0312,0.0064,EK__0_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,9.5,5.125,4.375,5.152000,0.001275,0.605590,0.000248,1175.428571,0.606709,5.142500,1.536576,0.428886,24.066265,4.813253,30.5,4.813293,3.177641,1.237874
2,1.0,11.6810,0.0827,0.0154,EK__1_0,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,19.0,10.250,8.750,11.684850,0.005150,0.707754,0.000441,1333.240000,0.708907,11.665850,1.447213,0.456716,59.728507,11.945701,30.0,11.945755,7.390361,2.878971
3,1.5,11.1076,0.1013,0.0191,EK__1_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,28.5,15.375,13.125,11.112375,0.003725,0.911596,0.000336,844.485714,0.913940,11.083875,1.564459,0.363639,59.870617,11.974123,29.5,11.975174,6.415281,2.499121
4,2.0,10.4497,0.1270,0.0302,EK__2_0,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,38.0,20.500,17.500,10.457250,0.009700,1.214468,0.000931,595.385714,1.218898,10.419250,1.689489,0.305452,58.470844,11.694169,29.0,11.700822,5.628702,2.192703


### Reducing operator dependence

In the tutorial, it was observed that blowcounts were always around 75 blows/m. This is because the hammer operator will try to keep the blowcount steady. A better target can therefore be selected when the number of blows is multiplied by the hammer energy. If the soil generates more resistance, this target will increase, even when the blowcount is steady. The following formula can be used:

$$ \text{Normalised total ENTHRU per distance} = \text{Blowcount} \cdot \text{Normalised ENTRHU} $$

It is easy to implement this with Pandas. Perform the calculation and stored the result in the colum ``Normalised total ENTRHU per distance [-/m]``.

In [14]:
data2['Normalised total ENTRHU per distance [-/m]'] = data2['Blowcount [Blows/m]'] * data2['Normalised ENTRHU [-]']

In [15]:
data2.head()

,z [m],qc [MPa],fs [MPa],u2 [MPa],ID,Location ID,Blowcount [Blows/m],Normalised ENTRHU [-],Normalised hammer energy [-],Number of blows,Diameter [m],Bottom wall thickness [mm],Pile penetration [m],area ratio [-],Push,Total unit weight [kN/m3],Layer no,Vertical total stress [kPa],Water pressure [kPa],Vertical effective stress [kPa],qt [MPa],Delta u2 [MPa],Rf [%],Bq [-],Qt [-],Fr [%],qnet [MPa],Ic [-],k,fsmax,fsres,h,fs,qaa,Qb,Normalised total ENTRHU per distance [-/m]
0,0.0,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,1.0,0.0,0.000,0.000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
1,0.5,5.1504,0.0312,0.0064,EK__0_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,9.5,5.125,4.375,5.152000,0.001275,0.605590,0.000248,1175.428571,0.606709,5.142500,1.536576,0.428886,24.066265,4.813253,30.5,4.813293,3.177641,1.237874,NaN
2,1.0,11.6810,0.0827,0.0154,EK__1_0,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,19.0,10.250,8.750,11.684850,0.005150,0.707754,0.000441,1333.240000,0.708907,11.665850,1.447213,0.456716,59.728507,11.945701,30.0,11.945755,7.390361,2.878971,NaN
3,1.5,11.1076,0.1013,0.0191,EK__1_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,28.5,15.375,13.125,11.112375,0.003725,0.911596,0.000336,844.485714,0.913940,11.083875,1.564459,0.363639,59.870617,11.974123,29.5,11.975174,6.415281,2.499121,NaN
4,2.0,10.4497,0.1270,0.0302,EK__2_0,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,38.0,20.500,17.500,10.457250,0.009700,1.214468,0.000931,595.385714,1.218898,10.419250,1.689489,0.305452,58.470844,11.694169,29.0,11.700822,5.628702,2.192703,NaN


### Shaft resistance proxy

The engineered feature for approximating shaft resistance can be calculated..

In [16]:
enhanced_data = pd.DataFrame()  # Create a dataframe for the data enhanced with the shaft friction feature

for location in data2['Location ID'].unique():  # Loop over all unique locations
    locationdata = data2[data2['Location ID'] == location].copy()  # Select the location-specific data
    # Calculate the shaft resistance feature
    locationdata["Qs [MN]"] = \
        (np.pi * locationdata["Diameter [m]"] * locationdata["z [m]"].diff() * locationdata["fs"]).cumsum()
    qaaf = locationdata['qaa'].values
    D = locationdata['Diameter [m]'].values
    t_mm = locationdata['Bottom wall thickness [mm]'].values

    # Convert thickness from mm to meters
    t = t_mm / 1000  # Convert from mm to meters

    # Calculate 'Qb' using NumPy based on the formula
    Qb = qaaf * np.pi * D * t

    # Add 'Qb' as a new column to the DataFrame
    locationdata['Qb'] = Qb
    locationdata['SRD [MN]'] = (locationdata["Qs [MN]"])
    enhanced_data = pd.concat([enhanced_data, locationdata])  # Combine data for the different locations in 1 dataframe

In [17]:
enhanced_data.head()

,z [m],qc [MPa],fs [MPa],u2 [MPa],ID,Location ID,Blowcount [Blows/m],Normalised ENTRHU [-],Normalised hammer energy [-],Number of blows,Diameter [m],Bottom wall thickness [mm],Pile penetration [m],area ratio [-],Push,Total unit weight [kN/m3],Layer no,Vertical total stress [kPa],Water pressure [kPa],Vertical effective stress [kPa],qt [MPa],Delta u2 [MPa],Rf [%],Bq [-],Qt [-],Fr [%],qnet [MPa],Ic [-],k,fsmax,fsres,h,fs,qaa,Qb,Normalised total ENTRHU per distance [-/m],Qs [MN],SRD [MN]
1,0.5,5.150400,0.031200,0.006400,EK__0_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,9.5,5.125,4.375,5.152000,0.001275,0.605590,0.000248,1175.428571,0.606709,5.142500,1.536576,0.428886,24.066265,4.813253,30.5,4.813293,3.177641,1.237874,NaN,NaN,NaN
2,1.0,11.681000,0.082700,0.015400,EK__1_0,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,19.0,10.250,8.750,11.684850,0.005150,0.707754,0.000441,1333.240000,0.708907,11.665850,1.447213,0.456716,59.728507,11.945701,30.0,11.945755,7.390361,2.878971,NaN,46.535583,46.535583
3,1.5,11.107600,0.101300,0.019100,EK__1_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,28.5,15.375,13.125,11.112375,0.003725,0.911596,0.000336,844.485714,0.913940,11.083875,1.564459,0.363639,59.870617,11.974123,29.5,11.975174,6.415281,2.499121,NaN,93.185771,93.185771
4,2.0,10.449700,0.127000,0.030200,EK__2_0,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,38.0,20.500,17.500,10.457250,0.009700,1.214468,0.000931,595.385714,1.218898,10.419250,1.689489,0.305452,58.470844,11.694169,29.0,11.700822,5.628702,2.192703,NaN,138.767197,138.767197
5,2.5,10.585762,0.113588,-0.116242,EK__2_5,EK,NaN,NaN,NaN,NaN,2.48,50.0,31.0,0.75,1.0,19.0,1.0,47.5,25.625,21.875,10.556701,-0.141867,1.075981,-0.013499,480.420639,1.080844,10.509201,1.679706,0.274977,60.975583,12.195117,28.5,12.214380,5.467241,2.129805,NaN,186.349231,186.349231


### Visualisation of new target variable

A plot can be created to show the variation of the new target variable vs cone tip resistance, depth and $ R_s $. The plot has three panels,

In [18]:
fig = subplots.make_subplots(rows=1, cols=3, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(
    x=enhanced_data["qc [MPa]"], y=enhanced_data["Normalised total ENTRHU per distance [-/m]"], showlegend=False,
    mode='markers',name='qc')
fig.append_trace(trace1, 1, 1)
trace2 = go.Scatter(
    x=enhanced_data["z [m]"], y=enhanced_data["Normalised total ENTRHU per distance [-/m]"], showlegend=False,
    mode='markers',name='z')
fig.append_trace(trace2, 1, 2)
trace3 = go.Scatter(
    x=enhanced_data["SRD [MN]"], y=enhanced_data["Normalised total ENTRHU per distance [-/m]"], showlegend=False,
    mode='markers',name='SRD')
fig.append_trace(trace3, 1, 3)

fig['layout']['xaxis1'].update(title='qc [MPa]', range=(0, 100), dtick=10)
fig['layout']['xaxis2'].update(title='z [m]', range=(0, 40), dtick=5)
fig['layout']['xaxis3'].update(title='SRD [MN]', range=(0, 12e3))
fig['layout']['yaxis1'].update(title='Normalised total ENTHRU [-/m]')

fig.show()

### Preparing data for machine learning

NaN values can be dropped:

In [19]:
enhanced_data.dropna(inplace=True)

### Train-splitting

A location-based train-test split is used. We will create a deep copy of the resulting dataframe. Otherwise, Pandas will update the original dataframe when we make changes, which is undesirable.

In [20]:
validation_ids = ['EL', 'CB', 'AV', 'BV', 'EF', 'DL', 'BM']
# Training data - ID not in validation_ids
training_data = deepcopy(enhanced_data[~enhanced_data['Location ID'].isin(validation_ids)])
# Validation data - ID in validation_ids
validation_data = deepcopy(enhanced_data[enhanced_data['Location ID'].isin(validation_ids)])

## Model building

### Support vector regression import

The SVR class can be imported (see scikit-learn docs).

In [21]:
from sklearn.svm import SVR

### SVR instance

An SVR instance can be created using the ``rbf`` kernel function and with other hyperparameters kept at their defaults. Can you identify from the documentation how these hyperparameters are determined and what their meaning is?

In [22]:
svr_rbf = SVR(kernel="rbf")

### Feature selection

The features on which we will train can be selected. Initially, we can work with a single feature ``Rs [kN]``. Can you add features to improve the accuracy of the model?

The target is our new normalised total ENTHRU.

In [23]:
features = ["SRD [MN]"]
X = training_data[features]
y = training_data["Normalised total ENTRHU per distance [-/m]"]

### Model training

We can train the model using ``SRD [MN]`` as the single feature and the total normalised ENTHRU as the target.

In [24]:
svr_rbf.fit(X, y)

SVR()

## Model predictions

### Training set

Making predictions is as easy as running the ``predict`` method on the trained model. We can assign the result to ``y_pred_train``.

In [25]:
y_pred_train = svr_rbf.predict(X)

The predictions can be visualised against the observed values.

In [26]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(
    x=enhanced_data["SRD [MN]"], y=enhanced_data["Normalised total ENTRHU per distance [-/m]"], showlegend=True,
    mode='markers',name='True values')
fig.append_trace(trace1, 1, 1)

trace1 = go.Scatter(
    x=X["SRD [MN]"], y=y_pred_train, showlegend=True,
    mode='markers',name='Predictions')
fig.append_trace(trace1, 1, 1)

fig['layout']['xaxis1'].update(title='SRD [MN]', range=(0, 12e3))
fig['layout']['yaxis1'].update(title='Normalised total ENTHRU [-/m]')

fig.show()

The $ R^2 $-score can be determined using the ``score`` method.

In [27]:
svr_rbf.score(X, y)

0.8683835622872998

### Test set

The features and target can be selected for the test set.

In [28]:
X_test = validation_data[features]
y_test = validation_data["Normalised total ENTRHU per distance [-/m]"]

Predictions can again be made.

In [29]:
y_pred_test = svr_rbf.predict(X_test)

And the $ R^2 $-score can be displayed. Does the model generalise well?

In [30]:
svr_rbf.score(X_test, y_test)

0.8814552495416815

It should be noted that the predictions are normalised total ENTRHU per unit length. To predict blowcount, we need to divide by the normalised ENTHRU. We can first assign the predictions to the column ``'Predictions'`` in the ``validation_data`` dataframe.

In [31]:
validation_data['Predictions'] = y_pred_test

The predicted blowcount can then be calculated in a single line of code.

In [32]:
validation_data['Predicted blowcount'] = validation_data['Predictions'] / validation_data['Normalised ENTRHU [-]']

### Results for selected locations

The results for selected locations in the test set can be visualised by overlaying predictions and observed values for the total normalised ENTRHU per unit length and blowcount.

The results show that the model generally performs well. However, the increasing detrimental effect of blowcount underprediction towards the final pile penetration is not captures in any of the scikit-learn metrics. Custom coding is required to get more insight in this.

In [33]:
# Available locations are 'EL', 'CB', 'AV', 'BV', 'EF', 'DL', 'BM'
selected_location = 'BM'

In [34]:
fig = subplots.make_subplots(rows=1, cols=2, print_grid=False, shared_yaxes=True)

loc_data = validation_data[validation_data['Location ID'] == selected_location]
trace_observations = go.Scatter(
    x=loc_data['Normalised total ENTRHU per distance [-/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed values')
fig.append_trace(trace_observations, 1, 1)

trace_predictions = go.Scatter(
    x=loc_data['Predictions'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predictions')
fig.append_trace(trace_predictions, 1, 1)

trace_observations_blct = go.Scatter(
    x=loc_data['Blowcount [Blows/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed blowcount')
fig.append_trace(trace_observations_blct, 1, 2)

trace_predictions_blct = go.Scatter(
    x=loc_data['Predicted blowcount'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predicted blowcount')
fig.append_trace(trace_predictions_blct, 1, 2)

fig['layout']['xaxis1'].update(title='Normalised total ENTRHU [-/m]', side='top', anchor='y')
fig['layout']['xaxis2'].update(title='Blowcount [Blows/m]', side='top', anchor='y')
fig['layout']['yaxis1'].update(title='Depth below mudline [m]', autorange='reversed')
fig['layout'].update(height=700, width=900)
fig.show()

## Modifications: Using Gridsearch to refine SVR

When the initial model performs as expected, you can investigate the effect of adding more features and changing the hyperparameters. Can you get even better scores by doing this.

In scikit-learn, there are methods such as ``GridSearchCV`` which allow selection of an optimal set of hyperparameters but discussing these methods is beyond the scope of this tutorial.

In [35]:
features = ["SRD [MN]"]
X = training_data[features]
y = training_data["Normalised total ENTRHU per distance [-/m]"]

In [36]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # List of C values to try
    'gamma': [0.01, 0.1, 1, 10, 100],  # List of gamma values to try
}

# Create an instance of the SVR model
svr_rbf = SVR(kernel="rbf")

# Create an instance of GridSearchCV
grid_search = GridSearchCV(svr_rbf, param_grid, cv=5)

# Fit the GridSearchCV instance to the data
grid_search.fit(X, y)

# Get the best hyperparameters and the best model
best_params = grid_search.best_params_
best_svr_rbf = grid_search.best_estimator_


In [37]:
print("Best C:", grid_search.best_params_['C'])
print("Best gamma:", grid_search.best_params_['gamma'])

Best C: 100
Best gamma: 100


In [38]:
y_pred_train = best_svr_rbf.predict(X)


In [39]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(
    x=enhanced_data["SRD [MN]"], y=enhanced_data["Normalised total ENTRHU per distance [-/m]"], showlegend=True,
    mode='markers',name='True values')
fig.append_trace(trace1, 1, 1)

trace1 = go.Scatter(
    x=X["SRD [MN]"], y=y_pred_train, showlegend=True,
    mode='markers',name='Predictions')
fig.append_trace(trace1, 1, 1)

fig['layout']['xaxis1'].update(title='SRD [MN]', range=(0, 12e3))
fig['layout']['yaxis1'].update(title='Normalised total ENTHRU [-/m]')

fig.show()

In [40]:
best_svr_rbf.score(X, y)

0.9640377501353742

## Test set

In [41]:
X_test = validation_data[features]
y_test = validation_data["Normalised total ENTRHU per distance [-/m]"]

In [42]:
y_pred_test = best_svr_rbf.predict(X_test)

And the $ R^2 $-score can be displayed. Does the model generalise well?

In [43]:
best_svr_rbf.score(X_test, y_test)

0.7751912982783933

In [44]:
validation_data['Predictions'] = y_pred_test

In [45]:
validation_data['Predicted blowcount'] = validation_data['Predictions'] / validation_data['Normalised ENTRHU [-]']

### Results for selected locations

The results for selected locations in the test set can be visualised by overlaying predictions and observed values for the total normalised ENTRHU per unit length and blowcount.

The results show that the model generally performs well. However, the increasing detrimental effect of blowcount underprediction towards the final pile penetration is not captures in any of the scikit-learn metrics. Custom coding is required to get more insight in this.

In [46]:
# Available locations are 'EL', 'CB', 'AV', 'BV', 'EF', 'DL', 'BM'
selected_location = 'BM'

In [47]:
fig = subplots.make_subplots(rows=1, cols=2, print_grid=False, shared_yaxes=True)

loc_data = validation_data[validation_data['Location ID'] == selected_location]
trace_observations = go.Scatter(
    x=loc_data['Normalised total ENTRHU per distance [-/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed values')
fig.append_trace(trace_observations, 1, 1)

trace_predictions = go.Scatter(
    x=loc_data['Predictions'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predictions')
fig.append_trace(trace_predictions, 1, 1)

trace_observations_blct = go.Scatter(
    x=loc_data['Blowcount [Blows/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed blowcount')
fig.append_trace(trace_observations_blct, 1, 2)

trace_predictions_blct = go.Scatter(
    x=loc_data['Predicted blowcount'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predicted blowcount')
fig.append_trace(trace_predictions_blct, 1, 2)

fig['layout']['xaxis1'].update(title='Normalised total ENTRHU [-/m]', side='top', anchor='y')
fig['layout']['xaxis2'].update(title='Blowcount [Blows/m]', side='top', anchor='y')
fig['layout']['yaxis1'].update(title='Depth below mudline [m]', autorange='reversed')
fig['layout'].update(height=700, width=900)
fig.show()

###Trying to improve the gridsearch

We will try to refine the gridsearch, the $ R^2 $-score  for training is larger than for testing, which indicates overfitting. We will refine the hyperparameters  C and gamma

In [48]:
features = ["SRD [MN]"]
X = training_data[features]
y = training_data["Normalised total ENTRHU per distance [-/m]"]

In [49]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# Define the parameter grid

param_grid = {
    'C': [0.01, 0.1, 1, 10],  # List of C values to try
    'gamma': [0.01, 0.1, 1, 10],  # List of gamma values to try
}

# Create an instance of the SVR model
svr_rbf = SVR(kernel="rbf")

# Create an instance of GridSearchCV
grid_search = GridSearchCV(svr_rbf, param_grid, cv=5)

# Fit the GridSearchCV instance to the data
grid_search.fit(X, y)

# Get the best hyperparameters and the best model
best_params = grid_search.best_params_
best_svr_rbf = grid_search.best_estimator_

In [50]:
print("Best C:", grid_search.best_params_['C'])
print("Best gamma:", grid_search.best_params_['gamma'])

Best C: 10
Best gamma: 0.01


In [51]:
y_pred_train = best_svr_rbf.predict(X)

In [52]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(
    x=enhanced_data["SRD [MN]"], y=enhanced_data["Normalised total ENTRHU per distance [-/m]"], showlegend=True,
    mode='markers',name='True values')
fig.append_trace(trace1, 1, 1)

trace1 = go.Scatter(
    x=X["SRD [MN]"], y=y_pred_train, showlegend=True,
    mode='markers',name='Predictions')
fig.append_trace(trace1, 1, 1)

fig['layout']['xaxis1'].update(title='SRD [MN]', range=(0, 12e3))
fig['layout']['yaxis1'].update(title='Normalised total ENTHRU [-/m]')

fig.show()

In [53]:
best_svr_rbf.score(X, y)

0.9194718648594092

###Test set

In [54]:
X_test = validation_data[features]
y_test = validation_data["Normalised total ENTRHU per distance [-/m]"]

In [55]:
y_pred_test = best_svr_rbf.predict(X_test)

Calculating the $ R^2 $-score

In [56]:
best_svr_rbf.score(X_test, y_test)

0.8593652646796717

In [57]:
validation_data['Predictions'] = y_pred_test

In [58]:
validation_data['Predicted blowcount'] = validation_data['Predictions'] / validation_data['Normalised ENTRHU [-]']

###Results from selected location

In [59]:
# Available locations are 'EL', 'CB', 'AV', 'BV', 'EF', 'DL', 'BM'
selected_location = 'BM'

In [60]:
fig = subplots.make_subplots(rows=1, cols=2, print_grid=False, shared_yaxes=True)

loc_data = validation_data[validation_data['Location ID'] == selected_location]
trace_observations = go.Scatter(
    x=loc_data['Normalised total ENTRHU per distance [-/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed values')
fig.append_trace(trace_observations, 1, 1)

trace_predictions = go.Scatter(
    x=loc_data['Predictions'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predictions')
fig.append_trace(trace_predictions, 1, 1)

trace_observations_blct = go.Scatter(
    x=loc_data['Blowcount [Blows/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed blowcount')
fig.append_trace(trace_observations_blct, 1, 2)

trace_predictions_blct = go.Scatter(
    x=loc_data['Predicted blowcount'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predicted blowcount')
fig.append_trace(trace_predictions_blct, 1, 2)

fig['layout']['xaxis1'].update(title='Normalised total ENTRHU [-/m]', side='top', anchor='y')
fig['layout']['xaxis2'].update(title='Blowcount [Blows/m]', side='top', anchor='y')
fig['layout']['yaxis1'].update(title='Depth below mudline [m]', autorange='reversed')
fig['layout'].update(height=700, width=900)
fig.show()

### Trying different combinations of hyperparameters C and gamma

In [61]:
features = ["SRD [MN]"]
X = training_data[features]
y = training_data["Normalised total ENTRHU per distance [-/m]"]

In [62]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# Define the parameter grid

param_grid = {
    'C': [0.1, 0.5, 1, 4],  # List of C values to try
    'gamma': [0.001, 0.1, 0.5]  # List of gamma values to try
}

# Create an instance of the SVR model
svr_rbf = SVR(kernel="rbf")

# Create an instance of GridSearchCV
grid_search = GridSearchCV(svr_rbf, param_grid, cv=5)

# Fit the GridSearchCV instance to the data
grid_search.fit(X, y)

# Get the best hyperparameters and the best model
best_params = grid_search.best_params_
best_svr_rbf = grid_search.best_estimator_

In [63]:
print("Best C:", grid_search.best_params_['C'])
print("Best gamma:", grid_search.best_params_['gamma'])

Best C: 4
Best gamma: 0.001


In [64]:
y_pred_train = best_svr_rbf.predict(X)

In [65]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(
    x=enhanced_data["SRD [MN]"], y=enhanced_data["Normalised total ENTRHU per distance [-/m]"], showlegend=True,
    mode='markers',name='True values')
fig.append_trace(trace1, 1, 1)

trace1 = go.Scatter(
    x=X["SRD [MN]"], y=y_pred_train, showlegend=True,
    mode='markers',name='Predictions')
fig.append_trace(trace1, 1, 1)

fig['layout']['xaxis1'].update(title='SRD [MN]', range=(0, 12e3))
fig['layout']['yaxis1'].update(title='Normalised total ENTHRU [-/m]')

fig.show()

In [66]:
best_svr_rbf.score(X, y)

0.8789258041242348

###Test set

In [67]:
X_test = validation_data[features]
y_test = validation_data["Normalised total ENTRHU per distance [-/m]"]

In [68]:
y_pred_test = best_svr_rbf.predict(X_test)

Calculating the $ R^2 $-score

In [69]:
best_svr_rbf.score(X_test, y_test)

0.8836116145266241

In [70]:
validation_data['Predictions'] = y_pred_test

In [71]:
validation_data['Predicted blowcount'] = validation_data['Predictions'] / validation_data['Normalised ENTRHU [-]']

###Results from selected location

In [72]:
# Available locations are 'EL', 'CB', 'AV', 'BV', 'EF', 'DL', 'BM'
selected_location = 'BM'

In [73]:
fig = subplots.make_subplots(rows=1, cols=2, print_grid=False, shared_yaxes=True)

loc_data = validation_data[validation_data['Location ID'] == selected_location]
trace_observations = go.Scatter(
    x=loc_data['Normalised total ENTRHU per distance [-/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed values')
fig.append_trace(trace_observations, 1, 1)

trace_predictions = go.Scatter(
    x=loc_data['Predictions'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predictions')
fig.append_trace(trace_predictions, 1, 1)

trace_observations_blct = go.Scatter(
    x=loc_data['Blowcount [Blows/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed blowcount')
fig.append_trace(trace_observations_blct, 1, 2)

trace_predictions_blct = go.Scatter(
    x=loc_data['Predicted blowcount'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predicted blowcount')
fig.append_trace(trace_predictions_blct, 1, 2)

fig['layout']['xaxis1'].update(title='Normalised total ENTRHU [-/m]', side='top', anchor='y')
fig['layout']['xaxis2'].update(title='Blowcount [Blows/m]', side='top', anchor='y')
fig['layout']['yaxis1'].update(title='Depth below mudline [m]', autorange='reversed')
fig['layout'].update(height=700, width=900)
fig.show()

The hyperparameter C trades off the correct classification of training examples agains maximization of the decision function's margin. C is a regularization parameter. A small value of C accepts a simpler decision function but with the trade-off of worse training accuracy.

The hyperparameter gamma controls how far the radius of influence of a single training sample reaches. In this case the best combination resulted in C=4 and gamma=0.001

##Implementing XGBoost Algorithm

XGBoost is an open source library that provides an efficient implementation of the gradient boosting algorithm, which is an ensemble machine learning algorithm that can be used for regression or classification problems.

Ensembles are constructed from decision tree models. Trees are added one at a time to the ensemble and fit to correct the prediction errors made by prior models. This is a type of ensemble machine learning model referred to as boosting.

In [74]:
#Changing column names since XGBoost does not accept names with special characters
training_data["SRD_MN"] = training_data["SRD [MN]"]
training_data["Normalised total ENTRHU per distance"] = training_data["Normalised total ENTRHU per distance [-/m]"]
features_new = ["SRD_MN"]

In [75]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score



features = ["SRD [MN]"]
X = training_data[features_new]
y = training_data["Normalised total ENTRHU per distance"]

# Create an XGBoost regression model
xgb_model = xgb.XGBRegressor()

# Fit the model to the training data
xgb_model.fit(X, y)

# Make predictions on the training data
y_train_pred = xgb_model.predict(X)

# Calculate Mean Squared Error (MSE) for training data
mse_train = mean_squared_error(y, y_train_pred)


# Calculate R-squared (R^2) for training  data
r2_train = r2_score(y, y_train_pred)


# Print MSE and R^2 for both training  data
print("Training MSE:", mse_train)

print("Training R-squared (R^2):", r2_train)











Training MSE: 17.83898378114014
Training R-squared (R^2): 0.9459332957667769


The predictions can be visualised against the observed values.

In [76]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(
    x=enhanced_data["SRD [MN]"], y=enhanced_data["Normalised total ENTRHU per distance [-/m]"], showlegend=True,
    mode='markers',name='True values')
fig.append_trace(trace1, 1, 1)

trace1 = go.Scatter(
    x=X["SRD_MN"], y=y_pred_train, showlegend=True,
    mode='markers',name='Predictions')
fig.append_trace(trace1, 1, 1)

fig['layout']['xaxis1'].update(title='SRD [MN]', range=(0, 12e3))
fig['layout']['yaxis1'].update(title='Normalised total ENTHRU [-/m]')

fig.show()

###Test set

In [77]:
#Changing column names since XGBoost does not accept names with special characters
validation_data["SRD_MN"] = validation_data["SRD [MN]"]
validation_data["Normalised total ENTRHU per distance"] = validation_data["Normalised total ENTRHU per distance [-/m]"]
features_new = ["SRD_MN"]

In [78]:
X_test = validation_data[features_new]
y_test = validation_data["Normalised total ENTRHU per distance"]

Predictions can be made

In [79]:
y_pred_test = xgb_model.predict(X_test)

And the $ R^2 $-score  and Mean Squared error (MSE) can be displayed.

In [80]:
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("Testing MSE:", mse_test)
print("Testing R-squared (R^2):", r2_test)

Testing MSE: 29.06450609070437
Testing R-squared (R^2): 0.8785530063114964


It should be noted that the predictions are normalised total ENTRHU per unit length. To predict blowcount, we need to divide by the normalised ENTHRU. We can first assign the predictions to the column ``'Predictions'`` in the ``validation_data`` dataframe.

In [81]:
validation_data['Predictions'] = y_pred_test

In [82]:
validation_data['Predicted blowcount'] = validation_data['Predictions'] / validation_data['Normalised ENTRHU [-]']

###Results for selected locations

The results for selected locations in the test set can be visualised by overlaying predictions and observed values for the total normalised ENTRHU per unit length and blowcount.

The results show that the model generally performs well. However, the increasing detrimental effect of blowcount underprediction towards the final pile penetration is not captures in any of the scikit-learn metrics. Custom coding is required to get more insight in this.

In [83]:
# Available locations are 'EL', 'CB', 'AV', 'BV', 'EF', 'DL', 'BM'
selected_location = 'BM'

In [84]:
fig = subplots.make_subplots(rows=1, cols=2, print_grid=False, shared_yaxes=True)

loc_data = validation_data[validation_data['Location ID'] == selected_location]
trace_observations = go.Scatter(
    x=loc_data['Normalised total ENTRHU per distance [-/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed values')
fig.append_trace(trace_observations, 1, 1)

trace_predictions = go.Scatter(
    x=loc_data['Predictions'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predictions')
fig.append_trace(trace_predictions, 1, 1)

trace_observations_blct = go.Scatter(
    x=loc_data['Blowcount [Blows/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed blowcount')
fig.append_trace(trace_observations_blct, 1, 2)

trace_predictions_blct = go.Scatter(
    x=loc_data['Predicted blowcount'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predicted blowcount')
fig.append_trace(trace_predictions_blct, 1, 2)

fig['layout']['xaxis1'].update(title='Normalised total ENTRHU [-/m]', side='top', anchor='y')
fig['layout']['xaxis2'].update(title='Blowcount [Blows/m]', side='top', anchor='y')
fig['layout']['yaxis1'].update(title='Depth below mudline [m]', autorange='reversed')
fig['layout'].update(height=700, width=900)
fig.show()

###Improving the model with gridsearch

In [85]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score

training_data["SRD_MN"] = training_data["SRD [MN]"]
training_data["Normalised total ENTRHU per distance"] = training_data["Normalised total ENTRHU per distance [-/m]"]
features_new = ["SRD_MN"]

features = ["SRD [MN]"]
X = training_data[features_new]
y = training_data["Normalised total ENTRHU per distance"]

# Create an XGBoost regression model
xgb_model = xgb.XGBRegressor()

# Define a parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.05, 0.1],
}

# Create an instance of GridSearchCV
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)

# Fit the GridSearchCV instance to the training data
grid_search.fit(X, y)

# Get the best hyperparameters and the best model
best_params = grid_search.best_params_
best_xgb_model = grid_search.best_estimator_


# Make predictions using the best model
y_train_pred = best_xgb_model.predict(X)

# Evaluate the model's performance
mse = mean_squared_error(y, y_train_pred)
r2 = r2_score(y, y_train_pred)

print("Best hyperparameters:", best_params)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R^2):", r2)


Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Mean Squared Error (MSE): 25.318754501678168
R-squared (R^2): 0.9232634757679942


In [86]:
y_pred_train = best_xgb_model.predict(X)

In [87]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(
    x=enhanced_data["SRD [MN]"], y=enhanced_data["Normalised total ENTRHU per distance [-/m]"], showlegend=True,
    mode='markers',name='True values')
fig.append_trace(trace1, 1, 1)

trace1 = go.Scatter(
    x=X["SRD_MN"], y=y_pred_train, showlegend=True,
    mode='markers',name='Predictions')
fig.append_trace(trace1, 1, 1)

fig['layout']['xaxis1'].update(title='SRD [MN]', range=(0, 12e3))
fig['layout']['yaxis1'].update(title='Normalised total ENTHRU [-/m]')

fig.show()

###Test set

In [88]:
#Changing column names since XGBoost does not accept names with special characters
validation_data["SRD_MN"] = validation_data["SRD [MN]"]
validation_data["Normalised total ENTRHU per distance"] = validation_data["Normalised total ENTRHU per distance [-/m]"]
features_new = ["SRD_MN"]

In [89]:
X_test = validation_data[features_new]
y_test = validation_data["Normalised total ENTRHU per distance"]

In [90]:
y_pred_test = best_xgb_model.predict(X_test)

And the $ R^2 $-score  and Mean Squared error (MSE) can be displayed.

In [91]:
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("Testing MSE:", mse_test)
print("Testing R-squared (R^2):", r2_test)

Testing MSE: 28.006926147244602
Testing R-squared (R^2): 0.88297213885473


In [92]:
validation_data['Predictions'] = y_pred_test

In [93]:
validation_data['Predicted blowcount'] = validation_data['Predictions'] / validation_data['Normalised ENTRHU [-]']

###Results from selected locations

In [94]:
# Available locations are 'EL', 'CB', 'AV', 'BV', 'EF', 'DL', 'BM'
selected_location = 'BM'

In [95]:
fig = subplots.make_subplots(rows=1, cols=2, print_grid=False, shared_yaxes=True)

loc_data = validation_data[validation_data['Location ID'] == selected_location]
trace_observations = go.Scatter(
    x=loc_data['Normalised total ENTRHU per distance [-/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed values')
fig.append_trace(trace_observations, 1, 1)

trace_predictions = go.Scatter(
    x=loc_data['Predictions'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predictions')
fig.append_trace(trace_predictions, 1, 1)

trace_observations_blct = go.Scatter(
    x=loc_data['Blowcount [Blows/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed blowcount')
fig.append_trace(trace_observations_blct, 1, 2)

trace_predictions_blct = go.Scatter(
    x=loc_data['Predicted blowcount'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predicted blowcount')
fig.append_trace(trace_predictions_blct, 1, 2)

fig['layout']['xaxis1'].update(title='Normalised total ENTRHU [-/m]', side='top', anchor='y')
fig['layout']['xaxis2'].update(title='Blowcount [Blows/m]', side='top', anchor='y')
fig['layout']['yaxis1'].update(title='Depth below mudline [m]', autorange='reversed')
fig['layout'].update(height=700, width=900)
fig.show()

###Gridsearch with regularization to reduce overfitting

In [96]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score

training_data["SRD_MN"] = training_data["SRD [MN]"]
training_data["Normalised total ENTRHU per distance"] = training_data["Normalised total ENTRHU per distance [-/m]"]
features_new = ["SRD_MN"]

features = ["SRD [MN]"]
X = training_data[features_new]
y = training_data["Normalised total ENTRHU per distance"]

# Create an XGBoost regression model
xgb_model = xgb.XGBRegressor()

# Define a parameter grid
param_grid = {
    'n_estimators': [50, 66],
    'max_depth': [3, 4],
    'learning_rate': [0.01, 0.1, 0.16],
    'reg_alpha': [0, 0.01, 0.15, 0.18, 1],        # Add alpha values for L1 regularization
    'reg_lambda': [0, 0.01, 0.15, 0.18, 1],       # Add lambda values for L2 regularization
}

# Create an instance of GridSearchCV
grid_search = GridSearchCV(xgb_model, param_grid, cv=10, scoring='neg_mean_squared_error', verbose=1)

# Fit the GridSearchCV instance to the training data
grid_search.fit(X, y)

# Get the best hyperparameters and the best model
best_params = grid_search.best_params_
best_xgb_model = grid_search.best_estimator_


# Make predictions using the best model
y_train_pred = best_xgb_model.predict(X)

# Evaluate the model's performance
mse = mean_squared_error(y, y_train_pred)
r2 = r2_score(y, y_train_pred)

print("Best hyperparameters:", best_params)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R^2):", r2)

Fitting 10 folds for each of 300 candidates, totalling 3000 fits
Best hyperparameters: {'learning_rate': 0.16, 'max_depth': 4, 'n_estimators': 66, 'reg_alpha': 1, 'reg_lambda': 0.01}
Mean Squared Error (MSE): 32.59419590382257
R-squared (R^2): 0.9012129406432435


In [97]:
y_pred_train = best_xgb_model.predict(X)

In [98]:
fig = subplots.make_subplots(rows=1, cols=1, print_grid=False, shared_yaxes=True)
trace1 = go.Scatter(
    x=enhanced_data["SRD [MN]"], y=enhanced_data["Normalised total ENTRHU per distance [-/m]"], showlegend=True,
    mode='markers',name='True values')
fig.append_trace(trace1, 1, 1)

trace1 = go.Scatter(
    x=X["SRD_MN"], y=y_pred_train, showlegend=True,
    mode='markers',name='Predictions')
fig.append_trace(trace1, 1, 1)

fig['layout']['xaxis1'].update(title='SRD [MN]', range=(0, 12e3))
fig['layout']['yaxis1'].update(title='Normalised total ENTHRU [-/m]')

fig.show()

###Test set

In [99]:
#Changing column names since XGBoost does not accept names with special characters
validation_data["SRD_MN"] = validation_data["SRD [MN]"]
validation_data["Normalised total ENTRHU per distance"] = validation_data["Normalised total ENTRHU per distance [-/m]"]
features_new = ["SRD_MN"]

In [100]:
X_test = validation_data[features_new]
y_test = validation_data["Normalised total ENTRHU per distance"]

In [101]:
y_pred_test = best_xgb_model.predict(X_test)

In [102]:
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("Testing MSE:", mse_test)
print("Testing R-squared (R^2):", r2_test)

Testing MSE: 28.117575490971728
Testing R-squared (R^2): 0.8825097869362998


In [103]:
validation_data['Predictions'] = y_pred_test

In [104]:
validation_data['Predicted blowcount'] = validation_data['Predictions'] / validation_data['Normalised ENTRHU [-]']

##Results in selected locations

In [105]:
# Available locations are 'EL', 'CB', 'AV', 'BV', 'EF', 'DL', 'BM'
selected_location = 'BM'

In [106]:
fig = subplots.make_subplots(rows=1, cols=2, print_grid=False, shared_yaxes=True)

loc_data = validation_data[validation_data['Location ID'] == selected_location]
trace_observations = go.Scatter(
    x=loc_data['Normalised total ENTRHU per distance [-/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed values')
fig.append_trace(trace_observations, 1, 1)

trace_predictions = go.Scatter(
    x=loc_data['Predictions'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predictions')
fig.append_trace(trace_predictions, 1, 1)

trace_observations_blct = go.Scatter(
    x=loc_data['Blowcount [Blows/m]'],
    y=loc_data['z [m]'], showlegend=True, mode='lines',name='Observed blowcount')
fig.append_trace(trace_observations_blct, 1, 2)

trace_predictions_blct = go.Scatter(
    x=loc_data['Predicted blowcount'],
    y=loc_data['z [m]'], showlegend=True, mode='markers',name='Predicted blowcount')
fig.append_trace(trace_predictions_blct, 1, 2)

fig['layout']['xaxis1'].update(title='Normalised total ENTRHU [-/m]', side='top', anchor='y')
fig['layout']['xaxis2'].update(title='Blowcount [Blows/m]', side='top', anchor='y')
fig['layout']['yaxis1'].update(title='Depth below mudline [m]', autorange='reversed')
fig['layout'].update(height=700, width=900)
fig.show()

###Summary

The $ R^2 $-score achieved with the algorithms that we tried are shown below:

\begin{array}{ccc}
\text{Algorithm}&\text{R-squared training}&\text{R-squared test}\\
Linregression&0.619&0.731\\
SVMrbf&0.868&0.882\\
XGBoost&0.901&0.882
\end{array}

We can conclude that the use of robust algorithms like Support Vector Machine with rbf and XGBoost, along with an improvement of the feature engineering based on domain knowledge allowed us to achieve a good result in the prediction of blow counts for pile driveability.